In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [3]:
data_full = [data_train, data_test]

In [4]:
data_train.shape

(891, 12)

In [5]:
data_test.shape

(418, 11)

In [6]:
data_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
data_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<h2>Adding Variable - Title</h2>

In [8]:
# A lot of the names given in the names of passengers we have certain titles like Mr, Mrs etc.

In [9]:
data_full[0].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
for df in data_full:
        df['Title']=0
        for i in df:
            df['Title']=df.Name.str.extract('([A-Za-z]+)\.')

In [11]:
data_full[0].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [12]:
# There are a few spelling mistakes in the titles of certain people. The following code is to rectify that:
for df in data_full:
    df['Title'].replace(['Mlle','Mme','Ms'],
                            ['Miss','Miss','Miss'],inplace=True)

In [13]:
# There are a few rare titles. The following code labels all of them as 'rare'
for df in data_full:
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 
                                            'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [14]:
data_full[0].Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rare'], dtype=object)

In [15]:
data_full[1].Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rare'], dtype=object)

In [16]:
for df in data_full:
    title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, "Master": 3, "Rare": 4}
    df['Title'] = df['Title'].map(title_mapping)

<h2>Age</h2>

In [17]:
# Age has a number of Null values in both train and test. This should be resolved first

In [18]:
# The titles that we found in the previous section could help in estimating the age of the null values
# Observing mean value of age of each of the titles above:
# In the train data:
data_full[0].groupby('Title')['Age'].mean() 

Title
0    32.368090
1    21.860000
2    35.898148
3     4.574167
4    45.545455
Name: Age, dtype: float64

In [19]:
# Using above data to fill in Null values 
data_train.loc[(data_train.Age.isnull())&(data_train['Title']==0),'Age']=32
data_train.loc[(data_train.Age.isnull())&(data_train['Title']==2),'Age']=36
data_train.loc[(data_train.Age.isnull())&(data_train['Title']==3),'Age']=5
data_train.loc[(data_train.Age.isnull())&(data_train['Title']==1),'Age']=22
data_train.loc[(data_train.Age.isnull())&(data_train['Title']==4),'Age']=46

In [20]:
data_full[1].groupby('Title')['Age'].mean() 

Title
0    32.000000
1    21.774844
2    38.903226
3     7.406471
4    43.833333
Name: Age, dtype: float64

In [21]:
data_test.loc[(data_test.Age.isnull())&(data_test['Title']==0),'Age']=32
data_test.loc[(data_test.Age.isnull())&(data_test['Title']==2),'Age']=39
data_test.loc[(data_test.Age.isnull())&(data_test['Title']==3),'Age']=7
data_test.loc[(data_test.Age.isnull())&(data_test['Title']==1),'Age']=22
data_test.loc[(data_test.Age.isnull())&(data_test['Title']==4),'Age']=44

In [22]:
for df in data_full:
#     df['Age_categorical']=0
    df.loc[df['Age']<=16,'Age']=0
    df.loc[(df['Age']>16)&(df['Age']<=32),'Age']=1
    df.loc[(df['Age']>32)&(df['Age']<=48),'Age']=2
    df.loc[(df['Age']>48)&(df['Age']<=64),'Age']=3
    df.loc[df['Age']>64,'Age']=4

In [23]:
data_full[0].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,1.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,2.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,1.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,2.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,2.0,0,0,373450,8.0500,NaN,S,0


<h2>Family Size</h2>

In [24]:
# This will combine the SibSp and Parch variables into one

In [25]:
for df in data_full:
    df['Family_Size']=0
    df['Family_Size'] = df['SibSp'] + df['Parch']

In [26]:
data_full[0].head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family_Size
0,1,0,3,"Braund, Mr. Owen Harris",male,1.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,2.0,1,0,PC 17599,71.2833,C85,C,2,1


<h2>Alone</h2>

In [27]:
# Binary Varibale indicating whether Alone or not 
# If Family_Size = 0  then Alone will be 1 

In [28]:
for df in data_full:
    df['Alone'] = 0
    df.loc[df.Family_Size==0,'Alone']=1

In [29]:
data_full[1].head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family_Size,Alone
0,892,3,"Kelly, Mr. James",male,2.0,0,0,330911,7.8292,NaN,Q,0,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,2.0,1,0,363272,7.0000,NaN,S,2,1,0


<h2>Sex</h2>

In [30]:
for df in data_full:
    df['Sex'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [31]:
data_full[1].head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family_Size,Alone
0,892,3,"Kelly, Mr. James",1,2.0,0,0,330911,7.8292,NaN,Q,0,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,2.0,1,0,363272,7.0000,NaN,S,2,1,0


<h2>Embarked</h2>

In [32]:
# There are two null values in Embarked in the train data. Since most of the embarked values are 'S' as can be seen
# EDA, I will fill them in with the value 'S'

In [33]:
data_full[0].loc[(data_full[0].Embarked.isnull()),'Embarked']= 'S'

In [34]:
#Now to map all embarked values to 0, 1, 2

In [35]:
for df in data_full:
    df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [36]:
data_full[0].head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family_Size,Alone
0,1,0,3,"Braund, Mr. Owen Harris",1,1.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2.0,1,0,PC 17599,71.2833,C85,1,2,1,0


<h2>Has_Cabin</h2>

In [37]:
for df in data_full:
    df['Has_Cabin'] = df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

In [38]:
data_test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family_Size,Alone,Has_Cabin
0,892,3,"Kelly, Mr. James",1,2.0,0,0,330911,7.8292,NaN,2,0,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,2.0,1,0,363272,7.0000,NaN,0,2,1,0,0
2,894,2,"Myles, Mr. Thomas Francis",1,3.0,0,0,240276,9.6875,NaN,2,0,0,1,0
3,895,3,"Wirz, Mr. Albert",1,1.0,0,0,315154,8.6625,NaN,0,0,0,1,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,1.0,1,1,3101298,12.2875,NaN,0,2,2,0,0


<h2>Fare</h2>

In [39]:
# There is one null value for Fare in test data set. We will this with the mean value 

In [40]:
data_full[1].Fare.mean()

35.6271884892086

In [41]:
data_test.loc[data_test.Fare.isnull(), 'Fare']= 36

In [42]:
# Turning Fare into categorical variable

In [43]:
for df in data_full:
    df.loc[ df['Fare'] <= 7.91, 'Fare']= 0
    df.loc[(df['Fare'] > 7.91) & (df['Fare'] <= 14.454), 'Fare'] = 1
    df.loc[(df['Fare'] > 14.454) & (df['Fare'] <= 31), 'Fare']   = 2
    df.loc[ df['Fare'] > 31, 'Fare']= 3
    df['Fare'] = df['Fare'].astype(int)

<h2>Removing Unimportant Variables</h2>

In [44]:
for df in data_full:
    df.drop(['Name','Ticket','Cabin',],axis=1,inplace=True)

In [45]:
data_full[1].head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_Size,Alone,Has_Cabin
0,892,3,1,2.0,0,0,0,2,0,0,1,0
1,893,3,0,2.0,1,0,0,0,2,1,0,0
2,894,2,1,3.0,0,0,1,2,0,0,1,0
3,895,3,1,1.0,0,0,1,0,0,0,1,0
4,896,3,0,1.0,1,1,1,0,2,2,0,0


In [46]:
data_full[0].head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_Size,Alone,Has_Cabin
0,1,0,3,1,1.0,1,0,0,0,0,1,0,0
1,2,1,1,0,2.0,1,0,3,1,2,1,0,1
2,3,1,3,0,1.0,0,0,1,0,1,0,1,0
3,4,1,1,0,2.0,1,0,3,0,2,1,0,1
4,5,0,3,1,2.0,0,0,1,0,0,0,1,0


In [47]:
data_train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
Family_Size    0
Alone          0
Has_Cabin      0
dtype: int64

In [48]:
data_test.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
Family_Size    0
Alone          0
Has_Cabin      0
dtype: int64

In [49]:
data_test.to_csv('test_clean.csv')

In [50]:
data_train.to_csv('train_clean.csv')